Code portions:
- All Pale chapters (through 23.1)
- Reasonably "optimal" preprocessing parameters
- fine-tuned embedding model
- ElasticSearch container Document Store
- Iterate and add other series embeddings?

In [4]:
%%bash

nvidia-smi


Sun Mar  5 21:55:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 516.94       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  ERR!                On   | 00000000:01:00.0 Off |                  N/A |
| N/A   42C    P0    N/A /  N/A |      0MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import os
import pickle
import logging
import time
from haystack.document_stores import ElasticsearchDocumentStore
from haystack import Document
from haystack.nodes import PreProcessor, EmbeddingRetriever, Seq2SeqGenerator, TransformersSummarizer, FARMReader
from haystack.pipelines import GenerativeQAPipeline, ExtractiveQAPipeline, SearchSummarizationPipeline
from haystack.utils import launch_es


In [6]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.WARNING)


In [7]:
# print(os.getcwd())
# os.chdir('./drive/MyDrive/pale-companion-files/es-experimentation/')

# print(os.getcwd())

In [21]:
MODEL_LOCATION = "../../twig_otherverse_parahumans_adapted/"
PALE_CHAPTERS_FNAME = "../../data/chapter_fmt_list.pkl"

In [14]:
with open(PALE_CHAPTERS_FNAME,'rb') as f:
    all_chapters = pickle.load(f)
chapters = [i for i in all_chapters if int(i['meta']['arc_number']) < 3]
print(f"Testing with {len(chapters)} chapters")

Testing with 62 chapters


In [15]:
preprocessor = PreProcessor(
    split_by='word',
    split_length=250,
    split_overlap=20,

    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_respect_sentence_boundary=True,
    progress_bar=True, 
    add_page_number=True
)
docs = preprocessor.process(chapters)
print(f"We will be working with {len(docs)} documents from {len(all_chapters)} chapters")

Preprocessing:   0%|          | 0/62 [00:01<?, ?docs/s]

We will be working with 2140 documents from 307 chapters


In [12]:
# Recommended: Start Elasticsearch using Docker via the Haystack utility function
# from haystack.utils import launch_es
# This is what we would use if we're outside of Colab - instead we have to do something else
launch_es()
time.sleep(30)

Unable to find image 'elasticsearch:7.17.6' locally
7.17.6: Pulling from library/elasticsearch
3b65ec22a9e9: Pulling fs layer
8aa839ec4bcf: Pulling fs layer
d59e7399c065: Pulling fs layer
aaa821b5c79a: Pulling fs layer
6b43de0011f5: Pulling fs layer
d0e32e747aea: Pulling fs layer
9df7bf99d869: Pulling fs layer
6c29c988ba57: Pulling fs layer
c547bc2077a7: Pulling fs layer
aaa821b5c79a: Waiting
6b43de0011f5: Waiting
d0e32e747aea: Waiting
9df7bf99d869: Waiting
6c29c988ba57: Waiting
c547bc2077a7: Waiting
d59e7399c065: Verifying Checksum
d59e7399c065: Download complete
8aa839ec4bcf: Verifying Checksum
8aa839ec4bcf: Download complete
6b43de0011f5: Verifying Checksum
6b43de0011f5: Download complete
3b65ec22a9e9: Verifying Checksum
3b65ec22a9e9: Download complete
d0e32e747aea: Verifying Checksum
d0e32e747aea: Download complete
9df7bf99d869: Verifying Checksum
9df7bf99d869: Download complete
6c29c988ba57: Verifying Checksum
6c29c988ba57: Download complete
c547bc2077a7: Verifying Checksum
c547bc

b0baf7f0328c2d49b6e0281de3cba8d45a5aa9471ca5ee17bd5c37572c796804


In [17]:
# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")
print(host)

document_store = ElasticsearchDocumentStore(
    host=host,
    username="",
    password="",
    index="pale_index",
    embedding_field="emb",
    embedding_dim=768,
    excluded_meta_data=["emb"],
)

document_store.write_documents(docs)

localhost


In [22]:
retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model=MODEL_LOCATION,
    model_format="sentence_transformers",
    max_seq_len=500,
    progress_bar=True,
)


document_store.update_embeddings(retriever)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.nodes.retriever.dense:Init retriever using embeddings of model ../twig_otherverse_parahumans_adapted


HFValidationError: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: '../twig_otherverse_parahumans_adapted'.

In [ ]:
generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa", num_beams=8, max_length=500)
# Eventually with want to tweak the above
pipe = GenerativeQAPipeline(generator, retriever) # We specify the params later

In [ ]:
TEST_QUESTION = "Who is Avery Kelly?"

TEST_OUTPUT = pipe.run(
                query = TEST_QUESTION,
                params = {
                    "Retriever": {"top_k":10},
                    # "Generator": {"max_length": maxlength, "num_beams": nbeam}
                }
            )

In [ ]:
TEST_OUTPUT